## **Maestría en Inteligencia Artificial Aplicada**
### **Curso: Inteligencia Artificial y Aprendizaje Automático**
#### Tecnológico de Monterrey
####Prof Luis Eduardo Falcón Morales

### **Adtividad de la Semana 8**
#### **Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Esta Actividad deberás resolverla de manera individual.**

**Nombre y matrícula:**

*   Aldo Daniel Nava Barrios
*   A01333185


In [149]:
# Incluye aquí todas las librerías y paquetes que requieras.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR

from sklearn import metrics
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import learning_curve
from sklearn.decomposition import TruncatedSVD


In [150]:
#Ahora asociamos el drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [151]:
#Seleccionamos el directorio
import os
DIR = "/content/drive/MyDrive/Maestría/Segundo trimestre/IA y ML/Actividad semana 8"
os.chdir(DIR)

### **Liga de Datos de la UCI "Restaurant & consumer data".**
### **Descarga "RCdata.zip" y de ahí necesitaremos los archivos "rating_final.csv" y "geoplaces2.csv".**

https://archive.ics.uci.edu/dataset/232/restaurant+consumer+data



In [152]:
# Descarga los archivos de la página de la UCI a partir de los cuales generaremos
# nuestras matrices de utilidad.

data_1 = pd.read_csv('rating_final.csv', sep=';', header='infer')
data_2 =pd.read_csv('geoplaces2.csv', header='infer',  encoding='latin-1')
print(data_1.shape, data_2.shape)

(1161, 1) (130, 21)


## **Ejercicio - 1**

### **Explica cuál es el propósito de usar el argumento "latin-1" al cargar el segundo archivo.**

El argumento "latin-1" se utiliza al cargar el segundo archivo (geoplaces2.csv) para especificar la codificación de caracteres del archivo. "latin-1" es una codificación de carcteres que se utiliza para representar caracteres en español. Utilizamos este argumento debido a que el Excel cuenta con nombres en español; si especificamos esto, se evitan errores en la codificación.

In [153]:
# Del primer archivo obtenemos una matriz con 3 evaluaciones de los restaurantes:
# general, comida y servicio.
# Las evaluaciones pueden ser 0, 1 o 2, siendo 0 la menor calificación y 2 la
# mayor calificación:

data_1.head()

,"userID,placeID,rating,food_rating,service_rating"
0,"U1077,135085,2,2,2"
1,"U1077,135038,2,2,1"
2,"U1077,132825,2,2,2"
3,"U1077,135060,1,2,2"
4,"U1068,135104,1,1,2"


In [154]:
# Del segundo archivo obtenemos información diversa de cada restaurante:

data_2.head(2)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none


In [155]:
# De data1 no requeriremos "rating" y de data2 solo necesitamos "placeID" y "name":
# Definimos la lista y matrices con los factores que necesitamos:
# Quiero aclarar que tuve que hacer otro dataframe porque las listas
# me ocasionaban un problema con la columna "placeID"; no obstante,
# modifiqué el códigopropuesto para que en lugar de listas fueran datasets.
nueva_data1 = pd.read_csv("rating_final.csv", usecols=lambda x: x != 'rating')
nueva_data_2 = data_2[['placeID', 'name']]


In [156]:
nueva_data1

,userID,placeID,food_rating,service_rating
0,U1077,135085,2,2
1,U1077,135038,2,1
2,U1077,132825,2,2
3,U1077,135060,2,2
4,U1068,135104,1,2
...,...,...,...,...
1156,U1043,132630,1,1
1157,U1011,132715,1,0
1158,U1068,132733,1,0
1159,U1068,132594,1,1


In [157]:
nueva_data_2

,placeID,name
0,134999,Kiku Cuernavaca
1,132825,puesto de tacos
2,135106,El Rincón de San Francisco
3,132667,little pizza Emilio Portes Gil
4,132613,carnitas_mata
...,...,...
125,132866,Chaires
126,135072,Sushi Itto
127,135109,Paniroles
128,135019,Restaurant Bar Coty y Pablo


## **Ejercicio - 2**

In [158]:
# Define el DataFrame que conjunte la información de las dos DataFrame data1a
# y data2a en uno solo, a través de la columna común "placeID" y que
# llamaremos "df_combinado":
df_combinado = pd.merge(nueva_data1, nueva_data_2, on='placeID')
# Despleguemos la dimensión y los primeros renglones de este DataFrame:
print(df_combinado.shape)
df_combinado.head()

(1161, 5)


,userID,placeID,food_rating,service_rating,name
0,U1077,135085,2,2,Tortas Locas Hipocampo
1,U1108,135085,2,1,Tortas Locas Hipocampo
2,U1081,135085,2,1,Tortas Locas Hipocampo
3,U1056,135085,2,2,Tortas Locas Hipocampo
4,U1134,135085,1,2,Tortas Locas Hipocampo


## **Ejercicio - 3**

In [159]:
#    Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación de la comida (food_rating). La llamaremos "UtMx_food".


# ************* Inlcuye aquí tu código:*****************************


UtMx_food = df_combinado.pivot_table(values='food_rating', index='name', columns='userID')


# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación de la comida:')
print('(restaurantes, usuarios) =', (UtMx_food.shape))
UtMx_food.head()

Dimensión de la matriz de Utilidad sobre la evaluación de la comida:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arrachela Grill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabana Huasteca,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
Cafe Chaires,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
Cafeteria cenidet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## **Ejercicio - 4**

### **En la Factorización SVD la cantidad de valores singulares será menor o igual al menor valor de los renglones o columnas de la matriz de utilidad UtMx.**

### **La factorización SVD de una matriz $A$ tiene la forma:**

$A_{m\times n} = U_{m\times m}\Sigma_{m\times n}V_{n\times n}^T$

In [160]:
# Apliquemos la transformación SVD para obtener la matriz de las variables
# latentes de los restaurantes en relación a la evalución de los usuarios con
# respecto a la comida. Deberás obtener la factorización SVD con el máximo de
# comonentes y por lo tanto el máximo de valores singulares posible.
# Asigna a la variable "nc_food" el valor del máximo número de valores singulares
# que podemos obtener de la utilidad que generamos en el ejercicio anterior.

# ************* Inlcuye aquí tu código:*****************************
m, n =UtMx_food.shape
nc_food= min(m, n)

#Quiero aclarar que la matriz de utilidad contiene valores NaN, los cuales no son
#compatibles directamente con la factorización SVD de TruncatedSVD, por lo tanto
#utilizamos UtMx_food_llena para reemplazar los NaN por un valor fijo
UtMx_food_llena = UtMx_food.fillna(UtMx_food.mean(axis=0))

SVD_food = TruncatedSVD( n_components = nc_food )
SVD_food.fit(UtMx_food_llena)
# *********** Aquí termina la sección de agregar código *************

# Inicializamos y aplicamos la factorización SVD en relación a la
# evaluación de la comida, para determinar la cantidad de componentes
# que podemos utilizar con un 90% de explicación de la variabilidad
# de dichas componentes:

# Determinemos la cantidad de componentes que explican un 90%
# de la variabilidad acumulada de cada componente:

for j in range(nc_food):
  if SVD_food.explained_variance_ratio_[0:j].sum() > 0.90:
    break

# Usaremos esta cantidad de componentes para las recomendaciones basadas
# en la calificación de la comida:
N_food = j-1

print('Total de valores singulares basados en evaluación de la :', nc_food)
print('Valor de truncamiento al 90% de dicha variabilidad:',N_food)

Total de valores singulares basados en evaluación de la : 129
Valor de truncamiento al 90% de dicha variabilidad: 54


In [161]:
# Usando la métrica de correlación de Pearson, obtendremos las 10 mejores
# recomendaciones de este modelo no supervisado con base a la información de
# alguien que desea obtener las similidades com el restaurante "Restaurante Pueblo
# Bonito" y con la cota de truncamiento obtenido en el ejercicio anterior.
# Para ello obtendremos las correlaciones positivas y de ahí deberás seleccionar
# las diez mejores.

# Factorización SVD:
SVD_food = TruncatedSVD(n_components = N_food)
resultant_matrix_food = SVD_food.fit_transform(UtMx_food_llena)

# Matriz de correlación de Pearson:
corr_mat_food = np.corrcoef(resultant_matrix_food)

# Restaurante de referencia:
restaurante_de_referencia = "Restaurante Pueblo Bonito"
nombres_rest = UtMx_food.T.columns  # nombres de restaurantes
idx_rest = list(nombres_rest).index(restaurante_de_referencia)
corr_rest = corr_mat_food[idx_rest] # Vector de Correlación del RinconHuasteco

# Buscando las correlaciones positivas:
idx = (corr_rest>0)
mejores_sim_food = list()
for i in range(len(nombres_rest[idx])):
  mejores_sim_food.append((corr_rest[idx][i], nombres_rest[idx][i]))

print('Total de similaridades positivas encontradas:', len(mejores_sim_food))

print('Algunos de los resultados encontrados:')
mejores_sim_food[0:10]

Total de similaridades positivas encontradas: 129
Algunos de los resultados encontrados:


[(0.9732515806643712, 'Abondance Restaurante Bar'),
 (0.9883176778621804, 'Arrachela Grill'),
 (0.9857500148395536, 'Cabana Huasteca'),
 (0.9809453492611171, 'Cafe Chaires'),
 (0.9913130562067404, 'Cafeteria cenidet'),
 (0.963644181118312, 'Cafeteria y Restaurant El Pacifico'),
 (0.9890916127198739, 'Carls Jr'),
 (0.9873492827159539, 'Carnitas Mata  Calle 16 de Septiembre'),
 (0.9876924899428496, 'Carreton de Flautas y Migadas'),
 (0.9855962684524899, 'Cenaduria El RincÃ³n de Tlaquepaque')]

## **Ejercicio - 5**

In [162]:
# Ordenar la lista de recomendaciones "mejores_sim_food" encontrada en el paso
# anterior de manera descendente y llamaremos "mejores_sim_food_ordenadas".
#
# NOTA: Puedes incluir a continuación las líneas de código que requieras.

# ************* Inlcuye aquí tu código:*****************************
mejores_sim_food_ordenadas = sorted(mejores_sim_food, key=lambda x: x[0], reverse=True)
# *********** Aquí termina la sección de agregar código *************
# Desplegamos las 10 mejores similitudes encontradas de manera descendente:
print('Similitudes con base a la evaluación de la comida con mayores valores de correlación:')
for k in range(1,11):
  print('%d> %s' % (k, mejores_sim_food_ordenadas[k]))

Similitudes con base a la evaluación de la comida con mayores valores de correlación:
1> (0.9924173647918842, 'La Virreina')
2> (0.9917965473775778, 'sirloin stockade')
3> (0.9917495107641595, 'churchs')
4> (0.9917417007846259, 'la Cochinita Pibil Restaurante Yucateco')
5> (0.9917337789326761, 'TACOS EL GUERO')
6> (0.991700066935941, 'Restaurant El Muladar de Calzada')
7> (0.9916827902906061, 'el pueblito')
8> (0.9916179411228546, 'tacos de la estacion')
9> (0.991578047715008, 'Log Yin')
10> (0.9915381915416198, 'Giovannis')


## **Ejercicio - 6**

### **De manera análoga y usando ahora la evaluación con respecto al servicio (service_rating), encuentrarás ahora los diez restaurantes con mayor similaridad al "Restaurante Pueblo Bonito" y con respecto a la métrica de similaridad de Pearson.**

### **Deberás usar la cantidad de componentes necesarias para que la varabilidad de la factorización SVD quede explicada en un 90%.**

### **a.  Para ello define primero la matriz de utilidad, UtMx_service, donde los renglones son los nombres de los restaurantes, las columnas los usuarios y las entradas las evaluaciones con respecto al servicio recibido en el restaurante.**





In [163]:
# a. Define la matriz de utilidad cuyos renglones sean los nombres de los
#    restaurantes, las columnas los IDs de los usuarios y las entradas la
#    evaluación del servicio (service_rating). La llamaremos "UtMx_service".


# ************* Inlcuye aquí tu código:*****************************
UtMx_service = df_combinado.pivot_table(values='service_rating', index='name', columns='userID')
#Quiero aclarar que la matriz de utilidad contiene valores NaN, los cuales no son
#compatibles directamente con la factorización SVD de TruncatedSVD, por lo tanto
#utilizamos UtMx_food_llena para reemplazar los NaN por un valor fijo
UtMx_service_llena = UtMx_service.fillna(UtMx_service.mean(axis=0))
# *********** Aquí termina la sección de agregar código *************


print('Dimensión de la matriz de Utilidad sobre la evaluación del servicio:')
print('(restaurantes, usuarios) =', (UtMx_service_llena.shape))
UtMx_service.head()

Dimensión de la matriz de Utilidad sobre la evaluación del servicio:
(restaurantes, usuarios) = (129, 138)


userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
name,,,,,,,,,,,,,,,,,,,,,
Abondance Restaurante Bar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arrachela Grill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cabana Huasteca,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
Cafe Chaires,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
Cafeteria cenidet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
# b. Incluye el código necesario para obtener y desplegar los 10 restaurantes con
#    mayor similaridad de Pearson al "Restaurante Pueblo Bonito", y usando solo
#    el 90% de las componentes que explican la varianza de la matriz UtMx_service.
#    Desplegarlos en orden descendente, sin incliur el restaurante de referencia.
# ************* Inlcuye aquí tu código:*****************************
# Cálculo del número de componentes que explican el 90% de la varianza
var_exp_service = np.var(UtMx_service_llena, axis=0) / np.sum(np.var(UtMx_service_llena, axis=0))
var_acum_service = np.cumsum(var_exp_service)

# Se verifica si hay un componente que explica el 90% de la varianza
if max(var_acum_service) < 0.9:
    print("No hay suficientes componentes para explicar el 90% de la varianza")
    exit()

# Se utiliza un bucle `while` para evitar errores al buscar el índice del componente
i = 0
while var_acum_service[i] < 0.9:
    i += 1
n_componentes_90_service = i + 1

# Factorización SVD
SVD_service = TruncatedSVD(n_components=n_componentes_90_service, random_state=42)
U_service = SVD_service.fit_transform(UtMx_service_llena)

# Se calcula la correlación a partir de la matriz UtMx_service_llena
corr_mat_service = np.corrcoef(UtMx_service_llena)

# Se define el Restaurante de referencia del ejercicio
restaurante_de_referencia = "Restaurante Pueblo Bonito"

# Se obtiene el índice del restaurante de referencia
nombres_rest = UtMx_service_llena.T.columns
idx_rest = list(nombres_rest).index(restaurante_de_referencia)

# Se calcula la correlación con el Restaurante Pueblo Bonito
corr_rest = corr_mat_service[idx_rest]

# Se seleccionan solo las correlaciones positivas con otros restaurantes, excluyendo el restaurante de referencia
idx = (corr_rest>0) & (nombres_rest != restaurante_de_referencia) & (nombres_rest != '')

# Se ordena la lista de correlaciones y nombres de restaurantes por la correlación
mejores_sim_service = sorted(zip(corr_rest[idx], nombres_rest[idx]), key=lambda x: x[0], reverse=True)

# Imprime las 10 mejores recomendaciones por correlación
for i in range(1, 11):
    if nombres_rest[idx][i-1] != restaurante_de_referencia and nombres_rest[idx][i-1] != '':
        # **Corrección:** Se limita la impresión a las 10 primeras recomendaciones
        if i <= 10:
            print(f"Recomendación {i}: {mejores_sim_service[i-1][1]} (Correlación: {mejores_sim_service[i-1][0]})")




# *********** Aquí termina la sección de agregar código *************

Recomendación 1: Rincon del Bife (Correlación: 0.9359881499322104)
Recomendación 2: Pollo_Frito_Buenos_Aires (Correlación: 0.9345909379461121)
Recomendación 3: tacos de barbacoa enfrente del Tec (Correlación: 0.9345268782862166)
Recomendación 4: la Cochinita Pibil Restaurante Yucateco (Correlación: 0.9339214498063438)
Recomendación 5: Restaurant and Bar and Clothesline Carlos N Charlies (Correlación: 0.9338020631120297)
Recomendación 6: Restaurant Teely (Correlación: 0.9333329035023332)
Recomendación 7: McDonalds Centro (Correlación: 0.9326628100204498)
Recomendación 8: churchs (Correlación: 0.9321162957053896)
Recomendación 9: rockabilly  (Correlación: 0.9319081196924657)
Recomendación 10: tortas hawai (Correlación: 0.931797353612207)


## **Ejercicio - 7**

### **Incluye tus comentarios y conclusiones de la actividad. En particular indica cuáles restaurantes encontraste con correlaciones altas tanto para la calificación de la comida, como del servicio.**

Como resultado de esta actividad, destaco tres grandes enseñanzas:
1. En primer lugar, al momento de obtener las matrices de utilidad es muy importante observar que no contengan valores NaN, ya que este tipo de valores no son compatibles directamente con la factorización SVD. Ignorar esto complica la programación.
2. Se debe tener mucho cuidado en especificar que únicamente se tomen en consideración las correlaciones positivas. En un par de ocasiones se me pasó y perdí mucho tiempo.
3. El código que explicó la varabilidad de la factorización SVD en un 90% fue mucho más difícil de procesar debido a que se tuvo que implementar un bucle `while` para evitar errores al buscar el índice del componente, al inicio, pensé que replicar el código de la instrucción 4 sería fácil, pero hay que tomar muchas consideraciones, que son explicadas en el ejercicio 6b.
4. Como hallazgo principal, los restaurantes presentes en el top 10 de correlaciones altas tanto en comida como en servicio son:
- la Cochinita Pibil Restaurante Yucateco con correlación de comida de 0.99 y de servicio de 0.93.
- churchs con correlación de comida de 0.99 y de servicio de 0.93.
Por lo tanto, serían los más recomendados para visitar.

>> **Fin de la Actividad de Sistemas de Recomendación**